# Web Interaction using Bokeh

In [8]:
#importing all the necessary libraries for bokeh interactive visualization
import logging
import pandas as pd
import numpy as np
from bokeh.io import output_file, show, curdoc, output_notebook, reset_output
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CategoricalColorMapper, HoverTool, WMTSTileSource
from bokeh.models.widgets import CheckboxGroup, CheckboxButtonGroup, Button, Select, Paragraph
from bokeh.layouts import widgetbox, row, column #,gridplot, Tabs, Panel\n",
from bokeh.transform import jitter
from bokeh.palettes import Spectral11
from bokeh import tile_providers
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.models import ColumnDataSource
import bokeh.models.tools
from bokeh.application.handlers.function import FunctionHandler
output_notebook()


Loading BokehJS ...

In [10]:
import pandas as pd
import numpy as np
pd.set_option("display.precision", 4)

#reading in data
kickstart = pd.read_csv("web_data.csv", index_col =0)
#create new column 'funded_perc'
kickstart['funded_perc'] = (kickstart['pledged'] / kickstart['goal'])

#removing all projects with more than 100 backers
kickstart_sub = kickstart['backers'] < 100
ks_sub = kickstart[kickstart_sub]


#creating bins for histogram
arr_hist, edges = np.histogram(kickstart['backers'], 
                               bins = int(100/5), 
                               range = [0, 100])


ks1 = pd.DataFrame({'backers': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})
ks1

#creating empty plot
p12 = figure(plot_height = 600, plot_width = 600, 
           title = 'Count of projects with a certain number of Backers',
          x_axis_label = 'backers', 
           y_axis_label = 'count')
#adding histogram to the empty plot
p12.quad(bottom=0, top=ks1['backers'], 
       left=ks1['left'], right=ks1['right'], 
       fill_color='blue', line_color='black')

show(p12)


arr_hist, edges = np.histogram(kickstart['goal'], 
                               bins = int(40000/4000), 
                               range = [0, 40000])

ks = pd.DataFrame({'goal': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})
ks


p = figure(plot_height = 600, plot_width = 600, 
           title = 'Count of projects with a certain Goal',
          x_axis_label = 'goal', 
           y_axis_label = 'count')

p.quad(bottom=0, top=ks['goal'], 
       left=ks['left'], right=ks['right'], 
       fill_color='green', line_color='black')
show(p)


from bokeh.plotting import figure
from bokeh.io import show

#creating range for x-axis
p3 = figure(x_range=["Art", "Comics", "Fashion", "Dance","Journalism","Photography","Publishing","Technology","Theater","Film & Video", "Music", "Food", "Crafts", "Games", "Design"])
#grouping goal by main category
y_bar = kickstart.groupby(by = 'main_category')['goal'].mean()

#plotting bar grpah
p3.vbar(x=["Art", "Comics", "Fashion", "Dance","Journalism","Photography","Publishing","Technology","Theater","Film & Video", "Music", "Food", "Crafts", "Games", "Design"], width=0.5, bottom=0, top=y_bar, color="yellow")
p3.xaxis.axis_label = "Main Categories"
p3.xaxis.axis_label_standoff = 30
p3.xaxis.major_label_orientation = "vertical"

show(p3)

In [11]:
#creating a new column 'month'
#converting 'launched' column to datetime format
kickstart['launched'] = pd.to_datetime(kickstart['launched'])
#extract month from date 
kickstart['month'] = kickstart['launched'].dt.month


from bokeh.plotting import figure
from bokeh.io import show

    
#implementing hover tool from Bokeh    
hover = HoverTool(tooltips = [('ks', '@f_interval'),
                             ('Num of projects with a certain Goal', '@f_goal')])



p4 = figure(x_range=["Jan", "Feb", "March", "April","May","June","July","Aug","Sept","Oct", "Nov", "Dec"])
y_bar = kickstart.groupby(by = 'month')['goal'].mean()

p4.vbar(x=["Jan", "Feb", "March", "April","May","June","July","Aug","Sept","Oct", "Nov", "Dec"], width=0.5, bottom=0, top=y_bar, color="blue")
p4.xaxis.axis_label = "Month"
p4.xaxis.axis_label_standoff = 30
p4.xaxis.major_label_orientation = "vertical"

show(p4)

In [19]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models.widgets import CheckboxGroup
from bokeh.models.widgets import PreText

#creating an html page to display the output
output_file("slider.html")

p1 = figure(plot_height = 600, plot_width = 600, 
           title = 'Count of projects with a certain Goal',
          x_axis_label = 'goal', 
           y_axis_label = 'count')
p1.quad(bottom=0, top=ks['goal'], 
       left=ks['left'], right=ks['right'], 
       fill_color='green', line_color='black')

p1.title.text_font_size = '20pt'

#adding a paragraph element (html element) to the webpage
p6 = Paragraph(text="""This is the graph showing the count of average goals for all projects""")

from bokeh.models.widgets import Slider, RangeSlider
# Slider to select the binwidth, value is selected number
binwidth_select = Slider(start = 0, end = 40000, 
                     step = 1, value = 10000,
                     title = 'goal')

# Update the plot when the value is changed
#binwidth_select.on_change('value', update)

# RangeSlider to change the maximum and minimum values on histogram
range_select = RangeSlider(start = 0, end = 40080, value = (0, 40000),
                           step = 5, title = 'goal')

# Update the plot when the value is changed
#range_select.on_change('value', update)

#create the layout for the first tab
layout4 = row(binwidth_select, p1, p6)
#add the layout to the first tab
tab1 = Panel(child=layout4, title= 'Goal')




select = Select(title="X-axis: ", value="backers", options=['none', 'main categories', 'backers'])
#select.on_change('value', update_func)

p = figure(plot_height = 600, plot_width = 600, 
           title = 'Count of projects with a certain number of Backers',
          x_axis_label = 'backers', 
           y_axis_label = 'count')
p.quad(bottom=0, top=ks1['backers'], 
       left=ks1['left'], right=ks1['right'], 
       fill_color='blue', line_color='black')

p7 = Paragraph(text="""This is the graph showing the count of average backers for all projects""")

p.title.text_font_size = '20pt'

lo5 = column(select, range_select)
layout5 = row(lo5, p, p7)
tab2 = Panel(child=layout5, title="Backers")



ks_sub['launched'] = pd.to_datetime(ks_sub['launched'])
ks_sub['month'] = ks_sub['launched'].dt.month

p4 = figure(x_range=["Jan", "Feb", "March", "April","May","June","July","Aug","Sept","Oct", "Nov", "Dec"], title= "Goal per month", width=600)
y_bar = ks_sub.groupby(by ='month')['goal'].mean()

p4.vbar(x=["Jan", "Feb", "March", "April","May","June","July","Aug","Sept","Oct", "Nov", "Dec"], width=0.5, bottom=0, top=y_bar, color="yellow")
p4.xaxis.axis_label = "Month"
p4.xaxis.axis_label_standoff = 30
p4.xaxis.major_label_orientation = "vertical"
p5 = Paragraph(text="""This is the graph showing the average goal per mmonth over the projects (per month)""")
#create the list of main categories to add to the checkbox
catslist = ks_sub['main_category'].unique().tolist()
#create checkbox
checkbox_cats = CheckboxGroup(labels=catslist, active=[])                               
layout3 = row(checkbox_cats, p4, p5)
p4.title.text_font_size = '20pt'
tab3 = Panel(child=layout3, title= 'Goal per month')


#merge all the tabs together
tabs = Tabs(tabs=[ tab1, tab2, tab3 ])

#display the tabs on the html page
show(tabs)



#show(widgetbox(dropdown))

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
